In [ ]:

import numpy as np
from generate_price_data import *
import pandas as pd
from constructing_models import *
from itertools import product

C:\Users\tobia\AppData\Roaming\Python\Python311\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\tobia\AppData\Roaming\Python\Python311\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\tobia\AppData\Roaming\Python\Python311\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please 

In [ ]:
datadir = Path.cwd().parent / "data"

EARLIEST_DATE = pd.Timestamp("2000-01-01")
MIN_ROW_COUNT = 4000

common_idx = None
c = 0
for sector in datadir.iterdir():
    for asset in sector.iterdir():
        df = pd.read_csv(datadir / asset, index_col=0)
        df.index = pd.to_datetime(df.index)
        df = df[df.index >= EARLIEST_DATE]
        common_idx = common_idx.intersection(df.index) if common_idx is not None else df.index
        c += 1

data = np.empty((len(common_idx), c))

c = 0
for sector in datadir.iterdir():
    for asset in sector.iterdir():
        df = pd.read_csv(datadir / asset, index_col=0)
        df.index = pd.to_datetime(df.index)
        df = df[df.index.isin(common_idx)]
        data[:, c] = df["Close"].pct_change()
        c += 1

data = data[1:, :]


In [ ]:
num_actions = 10
A = np.random.dirichlet(np.full(data.shape[1], 1), num_actions)

rb_capacity = [200, 500]
rb_sample_size = [5, 10]
gamma = [0.95]
t_replay_buffer = [100]
training_lookback_period = [15]
model_constructors = [construct_dqn_1, construct_dqn_2, construct_dqn_3]
target_update_freq = [50]
action_space = [A]
state_space = [data]

product_args = list(product(state_space, rb_capacity, rb_sample_size, action_space, gamma, target_update_freq, training_lookback_period, t_replay_buffer, model_constructors))
tester = Tester(*product_args)